In [1]:
import numpy as np
import pandas as pd
import requests
import os
import json
import pandas as pd
import time
import glob

sample_data = pd.read_csv("/home/phadke/ONR/ONR/data_collection/sample_data_Jan_17.csv", header=0)


In [2]:
sample_data.head()

,Unnamed: 0,Unnamed: 0.1,domain,id,id_str,party,screen_name,state
0,9580,9580,1.0,2858243286,2858243286,BJP,rkdugar04,Delhi
1,11357,11357,1.0,826082996943851520,826082996943851520,BJP,b_biddyut,Assam
2,61,61,1.0,947016857768169472,947016857768169472,BJP,wasimkh47913671,Maharashtra
3,8515,8515,1.0,2262967596,2262967596,BJP,sanjudwg,West Bengal
4,32491,32491,NaN,1176799649899982849,1176799649899982848,BJP,Pankajt91543058,To Be Added


In [3]:
valid_files = []
for idx, row in sample_data.iterrows():
    cid = str(row['id_str'])
    screen_name=row['screen_name']
    party=row['party']
    state=row['state']
    
    fname = "/home/phadke/ONR/ONR/big_data/Twitter/" + cid+"_"+screen_name+"_"+party+".csv"
    valid_files.append(fname)


In [4]:
print(len(valid_files))

4000


In [ ]:
linkframe = pd.DataFrame(columns=['party','link', 'author'])
counter=0
for d in valid_files:
    if counter%100==0:
        print(counter)
    counter+=1
    try:
        splitfilename = d.replace(".csv", "").split("_")
        party = splitfilename[len(splitfilename)-1]
        author = splitfilename[len(splitfilename)-2]
        with open(d, "r") as jsonfile:

            for line in jsonfile:
                job= json.loads(line)
                if "entities" in job.keys():
                    if "urls" in job["entities"]:
                        for u in job["entities"]["urls"]:
                            if "expanded_url" in u:
                                linkframe = linkframe.append({"party":party, "link":u["expanded_url"], "author":author}, ignore_index=True)
                                
    except:
        fakevar = 0


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600


In [ ]:
linkframe.head()

In [ ]:
#print(linkframe.head())

linkframe.to_csv("/home/phadke/ONR/ONR/lite_data/jan21_nonfiltered_extracted_urls.csv")
## extract domains

def get_domain(link):
    try:
        res = get_tld(link, as_object=True)
        return res.fld
    except:
        fakevar=1


linkframe['domain'] = linkframe['link'].parallel_apply(lambda x: get_domain(x))



In [ ]:
#print(linkframe.head())

common_domains = ['twitter.com', 'facebook.com', 'google.com', 'm.tech', 'm.sc', 'b.tech', 'page.link', 'youtu.be', 'bit.ly', 'instagram.com','youtube.com']

filtered_link = linkframe.loc[~linkframe['domain'].isin(common_domains)]

filtered_link.to_csv("/home/phadke/ONR/ONR/lite_data/jan21_filtered_extracted_urls.csv")

agg_filtered = filtered_link.groupby(['author','domain']).size().reset_index().rename(columns={0:"count"})

print(agg_filtered.head())

agg_filtered.to_csv("/home/phadke/ONR/ONR/lite_data/jan21_extracted_domains.csv")

In [ ]:
agg_filtered.sample(5)

In [ ]:
len(agg_filtered)